# shp2html

This script converts a shapefile with polygons to an HTML image map.

Input requirement:



### How to prepare Python environment

Requirement for python environment

1. Install native GDAL
    On Mac or unix-like system

    `sudo apt-get install gdal-bin`

    On macOS with homebrew

    `brew install gdal`

2. Create Python virtual environment 

    `virtualenv metpetools` 

3. Activate virtual environment

    `source ./bin/activate`

4. Install python dependencies

    `pip install GDAL`
    
    `pip install jupyter`


## RUN

After starting this cell go to the bottom of the cell and select the files with the widgets provided.


In [1]:
import json
import os
from zipfile import ZipFile
from osgeo import ogr
import ipywidgets as widgets
from IPython.display import display


Shapefile_selector = widgets.FileUpload(
    description='Shapefile',
    accept='.shp, .dbf, .shx, .cpg',  
    multiple=True  
)


Jpgfile_selector = widgets.FileUpload(
    description='Image',
    accept='',  
    multiple=False  
)
projectname = widgets.Text(
    description='Name:',
    placeholder='NAME'
)


labelShp = widgets.Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')
labelImg = widgets.Label(value='Select the Image file')
labelName = widgets.Label(value='Select the project name')
disclaimer = widgets.Label(value='Note: ouptut file will be saved as NAME.ZIP')
display(labelShp, Shapefile_selector, labelImg, Jpgfile_selector, labelName, projectname, disclaimer)





Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut file will be saved as NAME.ZIP')

In [2]:
from htmltemplate import HTMLHEAD
from htmltemplate import HTMLFOOT
from lib import polygoniterate
from lib import fix_coords
#from libshp2html import polygoniterate

NAME = projectname.value

SHAPEFILE = ''
# Get the uploaded files from the widgets
shapefiledata = iter(Shapefile_selector.value)
for filename in shapefiledata:
    with open(filename['name'], 'wb') as f:
        f.write(filename['content'])
    # set the shapefile name variable
    ShpFileName = filename['name']
    if ShpFileName.endswith('.shp'):
        SHAPEFILE = ShpFileName

# shapefile_data = next(iter(Shapefile_selector.value))
jpgfile_data = next(iter(Jpgfile_selector.value))

# Write the content of the file to disk
# with open(shapefile_data['name'], 'wb') as f:
#     f.write(shapefile_data['content'])

with open(jpgfile_data['name'], 'wb') as f:
    f.write(jpgfile_data['content'])


# set paths to the uploaded files
# SHAPEFILE = shapefiledata['name']
RASTER = jpgfile_data['name']

# Set a global variable that tracks the occurrence of inner rings in polygons
INRINGS = 0

# Set a global variable that holds the list of vertices of each polygon
polygons_vert = []
# set a global variable that holds the list of attributes of each polygon
attributes_list = []

# Open the shapefile

shapefile = ogr.Open(SHAPEFILE)

# Get the layer
layer = shapefile.GetLayer()


result = polygoniterate(layer, attributes_list, INRINGS, polygons_vert)
attributes_list = result[0] # get the attributes list
polygons_vert = result[1] # get the vertices list

image_block = f'<img src="{RASTER}" usemap="#image-map" '\
               'style="width: 100%; height: auto;">\n'

HTMLPOLYGONS = '<map name="image-map">\n'
for a, p in zip(attributes_list, polygons_vert):
    coordstr = str(p)[1:-1].replace('(','').replace(')','').replace(' ','')
    id_ = a['OBJECTID']
    line = f'\t<area target="_blank" id="imgzone{id_}" '\
           f'alt="Element #{id_}" title="Element #{id_}" '\
           f'coords="{coordstr}" shape="poly">\n'
    HTMLPOLYGONS += line
HTMLPOLYGONS += '</map>\n'

html_path = rf'{NAME}.html'
AttributeTable_path = rf'{NAME}.json'

# Save as file
with open(html_path, 'w', encoding="utf-8") as out_file:
    out_file.write(HTMLHEAD)
    out_file.write(image_block)
    out_file.write('\n')
    out_file.write(HTMLPOLYGONS)
    out_file.write(HTMLFOOT)

with open(f'{NAME}.json', 'w', encoding="utf-8") as f:
    json.dump(attributes_list, f)


# Create zip file
with ZipFile(rf'{NAME}.zip', 'w') as outzip:
    outzip.write(RASTER)
    outzip.write(html_path)
    outzip.write(AttributeTable_path)

# Remove files
os.remove(html_path)
os.remove(AttributeTable_path)
os.remove(RASTER)

# remove the uploaded files
shapefiledata = iter(Shapefile_selector.value)
for filename in shapefiledata:
    os.remove(filename['name'])
    # with open(filename['name'], 'wb') as f:
    #     f.write(filename['content'])
    # # set the shapefile name variable
    # ShpFileName = filename['name']
    # if ShpFileName.endswith('.shp'):
    #     SHAPEFILE = ShpFileName
    #     break


StopIteration: 